# Control Plane: Agent Fleet Management

----

This notebook demonstrates **batch registration** of agents/workflows and **real-time simulation** with live metrics tracking in Azure AI Foundry.

You will learn how to:

- **Batch Register Agents**: Create 10 agents at once for fleet management
- **Batch Register Workflows**: Create 10 workflows for multi-agent orchestration
- **Real-time Simulation**: Run continuous daemon simulations with live metrics
- **Sample Evaluation**: Evaluate agent performance on sample tasks 
- **Monitor in Portal**: View live metrics and traces in Azure AI Foundry Portal

* Code Reference Repository: [@guming3d, AI-Foundry-Agent-Simulation](https://github.com/guming3d/AI-Foundry-Agent-Simulation)

## Table of Contents

- [Setup](#setup)
- [Part 1: Batch Agent Registration](#part-1-batch-agent-registration)
- [Part 2: Batch Workflow Registration](#part-2-batch-workflow-registration)
- [Part 3: Real-time Daemon Simulation](#part-3-real-time-daemon-simulation)
- [Part 4: Sample Evaluations](#part-4-sample-evaluations)
- [Part 5: Portal Monitoring](#part-5-portal-monitoring)
- [Wrap-up](#wrap-up)

## Setup

This notebook reuses the configuration file (`.foundry_config.json`) created by `0_setup/1_setup.ipynb`.

- If the file is missing, run the setup notebook first.
- Make sure you can authenticate (e.g., `az login`), so `DefaultAzureCredential` can work.

In [ ]:
# Environment setup and PATH configuration
import json
import os
import subprocess
import asyncio
import threading
import time
import random
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

load_dotenv(override=True)

# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
possible_paths = [
    '/opt/homebrew/bin',   # macOS (Apple Silicon)
    '/usr/local/bin',      # macOS (Intel) / Linux
    '/usr/bin',            # Linux / Codespaces
    '/home/linuxbrew/.linuxbrew/bin',  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f'🔍 Azure CLI found: {result.stdout.strip()}')
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get('PATH', ''):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get('PATH', ''):
            paths_to_add.append(path)

if paths_to_add:
    os.environ['PATH'] = ':'.join(paths_to_add) + ':' + os.environ.get('PATH', '')
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print('✅ PATH looks good already')

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

In [ ]:
# Load Foundry project settings from .foundry_config.json
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity import DefaultAzureCredential

config_file = '../0_setup/.foundry_config.json'
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
except FileNotFoundError as e:
    print(f"⚠️ Could not find '{config_file}'.")
    print('💡 Run 0_setup/1_setup.ipynb first to create it.')
    raise e

FOUNDRY_NAME = config.get('FOUNDRY_NAME')
RESOURCE_GROUP = config.get('RESOURCE_GROUP')
LOCATION = config.get('LOCATION')
PROJECT_NAME = config.get('PROJECT_NAME', 'proj-default')
AZURE_AI_PROJECT_ENDPOINT = config.get('AZURE_AI_PROJECT_ENDPOINT')
AZURE_AI_MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME")

os.environ['FOUNDRY_NAME'] = FOUNDRY_NAME or ''
os.environ['LOCATION'] = LOCATION or ''
os.environ['RESOURCE_GROUP'] = RESOURCE_GROUP or ''
os.environ['AZURE_SUBSCRIPTION_ID'] = config.get('AZURE_SUBSCRIPTION_ID', '')

print(f"✅ Loaded settings from '{config_file}'.")
print(f"\n📌 Foundry name: {FOUNDRY_NAME}")
print(f"📌 Resource group: {RESOURCE_GROUP}")
print(f"📌 Location: {LOCATION}")
print(f"📌 Project endpoint: {AZURE_AI_PROJECT_ENDPOINT}")
print(f"📌 Model deployment: {AZURE_AI_MODEL_DEPLOYMENT_NAME}")

# Initialize credential for Azure services
credential = DefaultAzureCredential()

## Part 1: Batch Agent Registration

Register **10 agents** at once for fleet management. Each agent has a specific role (e.g., CustomerSupport, DataAnalyst, etc.) to simulate a realistic enterprise deployment.

| Agent Type | Description | Count |
|------------|-------------|-------|
| CustomerSupport | Handle customer inquiries | 2 |
| DataAnalyst | Analyze business data | 2 |
| TechSupport | Technical troubleshooting | 2 |
| SalesAssistant | Sales and product queries | 2 |
| GeneralAssistant | General purpose queries | 2 |

In [ ]:
# Define agent types with their instructions
AGENT_TYPES = [
    ("CustomerSupport", "You are a customer support agent. Help customers with inquiries, complaints, and account issues."),
    ("DataAnalyst", "You are a data analyst. Analyze data, create reports, and provide business insights."),
    ("TechSupport", "You are a technical support agent. Help with software issues, troubleshooting, and technical guidance."),
    ("SalesAssistant", "You are a sales assistant. Help with product information, pricing, and purchase recommendations."),
    ("GeneralAssistant", "You are a general assistant. Help with various tasks and questions."),
]

# Batch create 10 agents (2 per type)
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")
created_agents = []

print("=" * 70)
print("🚀 Batch Agent Registration (10 Agents)")
print("=" * 70)

with AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as project_client:
    agent_count = 0
    for agent_type, instructions in AGENT_TYPES:
        for i in range(2):  # 2 agents per type
            agent_name = f"fleet-{agent_type}-{uuid_suffix}-{i+1:02d}"
            try:
                agent = project_client.agents.create_version(
                    agent_name=agent_name,
                    definition=PromptAgentDefinition(
                        model=AZURE_AI_MODEL_DEPLOYMENT_NAME,
                        instructions=instructions,
                    ),
                )
                created_agents.append({
                    "name": agent.name,
                    "id": agent.id,
                    "type": agent_type,
                    "model": AZURE_AI_MODEL_DEPLOYMENT_NAME,
                })
                agent_count += 1
                print(f"   ✅ [{agent_count:02d}/10] Created: {agent_name}")
            except Exception as e:
                print(f"   ❌ Failed to create {agent_name}: {str(e)[:50]}")

print(f"\n📊 Summary: {len(created_agents)}/10 agents created successfully")
print(f"   Agent IDs saved for simulation")

## Part 2: Batch Workflow Registration

Register **10 workflows** to orchestrate multi-agent interactions. Each workflow defines a sequence of agent calls for complex tasks.

| Workflow Type | Description | Agents Used |
|---------------|-------------|-------------|
| CustomerJourney | End-to-end customer support | CustomerSupport → TechSupport |
| DataPipeline | Data analysis workflow | DataAnalyst → GeneralAssistant |
| SalesProcess | Sales funnel workflow | SalesAssistant → CustomerSupport |
| TechEscalation | Technical issue escalation | TechSupport → DataAnalyst |
| GeneralInquiry | General purpose workflow | GeneralAssistant → SalesAssistant |

In [ ]:
# Import WorkflowAgentDefinition for workflow registration
from azure.ai.projects.models import WorkflowAgentDefinition

# Define workflow types with patterns
WORKFLOW_TYPES = [
    ("CustomerJourney", "End-to-end customer support workflow", "sequential"),
    ("DataPipeline", "Data analysis and reporting pipeline", "sequential"),
    ("SalesProcess", "Sales funnel and conversion workflow", "sequential"),
    ("TechEscalation", "Technical issue escalation workflow", "review_loop"),
    ("GeneralInquiry", "General purpose inquiry workflow", "sequential"),
]

# Map workflow types to agent pairs (from created agents)
def get_agents_for_workflow(workflow_type: str, agents: list) -> tuple:
    """Get agent pair for a workflow based on type."""
    agent_map = {
        "CustomerJourney": ("CustomerSupport", "TechSupport"),
        "DataPipeline": ("DataAnalyst", "GeneralAssistant"),
        "SalesProcess": ("SalesAssistant", "CustomerSupport"),
        "TechEscalation": ("TechSupport", "DataAnalyst"),
        "GeneralInquiry": ("GeneralAssistant", "SalesAssistant"),
    }
    primary_type, secondary_type = agent_map.get(workflow_type, ("GeneralAssistant", "GeneralAssistant"))
    
    # Find matching agents
    primary = next((a for a in agents if a["type"] == primary_type), agents[0] if agents else None)
    secondary = next((a for a in agents if a["type"] == secondary_type), agents[-1] if agents else None)
    return primary, secondary

def build_sequential_workflow_yaml(primary_agent: str, secondary_agent: str) -> str:
    """Build a sequential workflow YAML with two agents."""
    return f"""kind: workflow
trigger:
  kind: OnConversationStart
  id: workflow_start
  actions:
    - kind: SetVariable
      id: set_variable_input
      variable: Local.LatestMessage
      value: "=UserMessage(System.LastMessageText)"
    - kind: CreateConversation
      id: create_primary_conversation
      conversationId: Local.PrimaryConversationId
    - kind: InvokeAzureAgent
      id: primary_agent
      description: Primary Agent
      conversationId: "=Local.PrimaryConversationId"
      agent:
        name: {primary_agent}
      input:
        messages: "=Local.LatestMessage"
      output:
        messages: Local.LatestMessage
    - kind: CreateConversation
      id: create_secondary_conversation
      conversationId: Local.SecondaryConversationId
    - kind: InvokeAzureAgent
      id: secondary_agent
      description: Secondary Agent
      conversationId: "=Local.SecondaryConversationId"
      agent:
        name: {secondary_agent}
      input:
        messages: "=Local.LatestMessage"
      output:
        messages: Local.FinalMessage
        autoSend: true"""

def build_review_loop_workflow_yaml(primary_agent: str, reviewer_agent: str) -> str:
    """Build a review loop workflow YAML with primary and reviewer agents."""
    return f"""kind: workflow
trigger:
  kind: OnConversationStart
  id: workflow_start
  actions:
    - kind: SetVariable
      id: set_variable_input
      variable: Local.LatestMessage
      value: "=UserMessage(System.LastMessageText)"
    - kind: SetVariable
      id: set_variable_turncount
      variable: Local.TurnCount
      value: "=0"
    - kind: CreateConversation
      id: create_primary_conversation
      conversationId: Local.PrimaryConversationId
    - kind: CreateConversation
      id: create_reviewer_conversation
      conversationId: Local.ReviewerConversationId
    - kind: InvokeAzureAgent
      id: primary_agent
      description: Primary Agent
      conversationId: "=Local.PrimaryConversationId"
      agent:
        name: {primary_agent}
      input:
        messages: "=Local.LatestMessage"
      output:
        messages: Local.LatestMessage
    - kind: InvokeAzureAgent
      id: reviewer_agent
      description: Reviewer Agent
      conversationId: "=Local.ReviewerConversationId"
      agent:
        name: {reviewer_agent}
      input:
        messages: "=Local.LatestMessage"
      output:
        messages: Local.LatestMessage
    - kind: SetVariable
      id: increment_turncount
      variable: Local.TurnCount
      value: "=Local.TurnCount + 1"
    - kind: ConditionGroup
      id: completion_check
      conditions:
        - condition: '=!IsBlank(Find("[COMPLETE]", Upper(Last(Local.LatestMessage).Text)))'
          id: check_done
          actions:
            - kind: EndConversation
              id: end_workflow
        - condition: "=Local.TurnCount >= 3"
          id: check_turn_count_exceeded
          actions:
            - kind: SendActivity
              id: send_final
              activity: "Review complete."
      elseActions:
        - kind: GotoAction
          id: goto_primary_agent
          actionId: primary_agent"""

# Batch create 10 workflows (2 per type) and register in Azure
created_workflows = []

print("=" * 70)
print("🔄 Batch Workflow Registration (10 Workflows)")
print("=" * 70)

if not created_agents:
    print("⚠️ No agents available. Run Part 1 first to create agents.")
else:
    with AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as project_client:
        workflow_count = 0
        for workflow_type, description, pattern in WORKFLOW_TYPES:
            for i in range(2):  # 2 workflows per type
                workflow_name = f"wf-{workflow_type}-{uuid_suffix}-{i+1:02d}"
                
                # Get agents for this workflow
                primary_agent, secondary_agent = get_agents_for_workflow(workflow_type, created_agents)
                
                if not primary_agent or not secondary_agent:
                    print(f"   ⚠️ Skipping {workflow_name}: No agents available")
                    continue
                
                try:
                    # Build workflow YAML based on pattern
                    if pattern == "review_loop":
                        workflow_yaml = build_review_loop_workflow_yaml(
                            primary_agent["name"],
                            secondary_agent["name"]
                        )
                    else:
                        workflow_yaml = build_sequential_workflow_yaml(
                            primary_agent["name"],
                            secondary_agent["name"]
                        )
                    
                    # Register workflow in Azure AI Foundry
                    workflow = project_client.agents.create_version(
                        agent_name=workflow_name,
                        definition=WorkflowAgentDefinition(workflow=workflow_yaml),
                    )
                    
                    workflow_config = {
                        "name": workflow.name,
                        "id": workflow.id,
                        "version": workflow.version,
                        "type": workflow_type,
                        "pattern": pattern,
                        "description": description,
                        "agents": [primary_agent["name"], secondary_agent["name"]],
                        "created_at": datetime.now().isoformat(),
                    }
                    created_workflows.append(workflow_config)
                    workflow_count += 1
                    print(f"   ✅ [{workflow_count:02d}/10] Registered: {workflow_name} ({pattern})")
                    
                except Exception as e:
                    print(f"   ❌ Failed to create {workflow_name}: {str(e)[:60]}")

    print(f"\n📊 Summary: {len(created_workflows)}/10 workflows registered in Azure AI Foundry")
    print(f"   Workflow patterns: sequential, review_loop")
    print(f"   Each workflow orchestrates 2 agents")

## Part 3: Real-time Daemon Simulation

Run a **continuous daemon simulation** that sends requests to agents and tracks live metrics. This simulates production traffic for monitoring and testing.

### Daemon Configuration

| Parameter | Value | Description |
|-----------|-------|-------------|
| Interval | 5 seconds | Time between batches |
| Calls per batch | 3-5 | Random calls per interval |
| Threads | 3 | Parallel execution threads |
| Duration | 60 seconds | Total simulation time |

In [ ]:
# Daemon Configuration
@dataclass
class DaemonConfig:
    """Configuration for daemon simulation."""
    interval_seconds: float = 5.0
    calls_per_batch_min: int = 3
    calls_per_batch_max: int = 5
    threads: int = 3
    duration_seconds: int = 60
    run_evaluations: bool = False  # Enable/disable evaluations during simulation
    evaluation_count: int = 10     # Number of evaluation runs

@dataclass
class DaemonMetrics:
    """Live metrics for daemon monitoring."""
    total_calls: int = 0
    successful_calls: int = 0
    failed_calls: int = 0
    total_latency_ms: float = 0
    evaluation_runs: int = 0
    start_time: Optional[datetime] = None
    
    @property
    def success_rate(self) -> float:
        return (self.successful_calls / self.total_calls * 100) if self.total_calls > 0 else 0
    
    @property
    def avg_latency_ms(self) -> float:
        return (self.total_latency_ms / self.successful_calls) if self.successful_calls > 0 else 0
    
    @property
    def runtime(self) -> str:
        if not self.start_time:
            return "0s"
        elapsed = (datetime.now() - self.start_time).total_seconds()
        return f"{int(elapsed)}s"

# Sample queries for each agent type
QUERY_TEMPLATES = {
    "CustomerSupport": ["How do I reset my password?", "I need help with my order", "What's your return policy?"],
    "DataAnalyst": ["Analyze Q4 sales data", "Show revenue trends", "Compare regional performance"],
    "TechSupport": ["My app crashes on startup", "How do I install the SDK?", "Network connection issues"],
    "SalesAssistant": ["What's the pricing for enterprise?", "Compare product features", "Request a demo"],
    "GeneralAssistant": ["What time is it in Tokyo?", "Summarize this article", "Help me draft an email"],
}

# Evaluation templates directory
EVALUATION_TEMPLATES_DIR = os.path.join(os.path.dirname(os.path.abspath(".")), "1_controlplane", "evaluation_templates")

print("✅ Daemon configuration ready")
print(f"   Interval: {DaemonConfig().interval_seconds}s")
print(f"   Calls/batch: {DaemonConfig().calls_per_batch_min}-{DaemonConfig().calls_per_batch_max}")
print(f"   Duration: {DaemonConfig().duration_seconds}s")
print(f"   Evaluations enabled: {DaemonConfig().run_evaluations}")
print(f"   Evaluation templates: {EVALUATION_TEMPLATES_DIR}")

In [ ]:
# Daemon Runner - Simulates continuous production traffic
from pathlib import Path
import yaml

@dataclass
class EvaluationItem:
    """Single evaluation dataset row."""
    query: str
    context: str = ""
    ground_truth: str = ""

@dataclass
class EvaluationResult:
    """Result of a single evaluation."""
    template_id: str
    agent_name: str
    query: str
    response: str
    success: bool
    latency_ms: float

class DaemonRunner:
    """Simple daemon runner for agent fleet simulation with optional evaluations."""
    
    def __init__(self, agents: List[Dict], config: DaemonConfig, templates_dir: str = None):
        self.agents = agents
        self.config = config
        self.metrics = DaemonMetrics()
        self._stop_requested = False
        self._lock = threading.Lock()
        self.templates_dir = Path(templates_dir) if templates_dir else Path("evaluation_templates")
        self.evaluation_results: List[EvaluationResult] = []
    
    def _load_evaluation_templates(self) -> List[Dict]:
        """Load evaluation templates from YAML files."""
        templates = []
        if not self.templates_dir.exists():
            print(f"   ⚠️ Templates dir not found: {self.templates_dir}")
            return templates
        
        for yaml_file in self.templates_dir.glob("*.yaml"):
            try:
                with open(yaml_file, 'r', encoding='utf-8') as f:
                    template = yaml.safe_load(f)
                    templates.append(template)
            except Exception as e:
                print(f"   ⚠️ Failed to load {yaml_file.name}: {e}")
        return templates
    
    def _call_agent(self, agent: Dict) -> Dict[str, Any]:
        """Execute a single agent call and return metrics."""
        agent_type = agent.get("type", "GeneralAssistant")
        query = random.choice(QUERY_TEMPLATES.get(agent_type, QUERY_TEMPLATES["GeneralAssistant"]))
        
        start_time = time.time()
        success = False
        error_msg = None
        
        try:
            with AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as client:
                openai_client = client.get_openai_client()
                conversation = openai_client.conversations.create()
                response = openai_client.responses.create(
                    conversation=conversation.id,
                    extra_body={"agent": {"name": agent["name"], "type": "agent_reference"}},
                    input=query,
                )
                success = True
        except Exception as e:
            error_msg = str(e)[:50]
        
        latency_ms = (time.time() - start_time) * 1000
        
        with self._lock:
            self.metrics.total_calls += 1
            if success:
                self.metrics.successful_calls += 1
                self.metrics.total_latency_ms += latency_ms
            else:
                self.metrics.failed_calls += 1
        
        return {"success": success, "latency_ms": latency_ms, "agent": agent["name"], "error": error_msg}
    
    def _run_evaluation(self, template: Dict, agent: Dict) -> EvaluationResult:
        """Run a single evaluation against an agent."""
        dataset_items = template.get("dataset", {}).get("items", [])
        if not dataset_items:
            return None
        
        item = random.choice(dataset_items)
        query = item.get("query", "")
        
        start_time = time.time()
        success = False
        response_text = ""
        
        try:
            with AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as client:
                openai_client = client.get_openai_client()
                conversation = openai_client.conversations.create()
                response = openai_client.responses.create(
                    conversation=conversation.id,
                    extra_body={"agent": {"name": agent["name"], "type": "agent_reference"}},
                    input=query,
                )
                response_text = getattr(response, 'output_text', str(response))[:200]
                success = True
        except Exception as e:
            response_text = f"Error: {str(e)[:50]}"
        
        latency_ms = (time.time() - start_time) * 1000
        
        with self._lock:
            self.metrics.evaluation_runs += 1
        
        return EvaluationResult(
            template_id=template.get("id", "unknown"),
            agent_name=agent["name"],
            query=query[:50],
            response=response_text[:100],
            success=success,
            latency_ms=latency_ms,
        )
    
    def _run_batch(self, batch_size: int):
        """Run a batch of agent calls in parallel."""
        threads = []
        for _ in range(batch_size):
            agent = random.choice(self.agents)
            t = threading.Thread(target=self._call_agent, args=(agent,))
            t.start()
            threads.append(t)
        for t in threads:
            t.join()
    
    def _run_evaluations_batch(self, templates: List[Dict], count: int):
        """Run evaluation batch."""
        print(f"\n   🧪 Running {count} evaluations...")
        for i in range(count):
            template = random.choice(templates)
            agent = random.choice(self.agents)
            result = self._run_evaluation(template, agent)
            if result:
                self.evaluation_results.append(result)
                status = "✅" if result.success else "❌"
                print(f"      {status} [{i+1}/{count}] {template.get('id', 'unknown')} → {agent['name'][:20]}...")
    
    def run(self):
        """Run the daemon simulation loop."""
        self.metrics.start_time = datetime.now()
        end_time = time.time() + self.config.duration_seconds
        batch_num = 0
        
        print("\n" + "=" * 70)
        print("🚀 Starting Daemon Simulation")
        print("=" * 70)
        
        # Load evaluation templates if enabled
        templates = []
        if self.config.run_evaluations:
            templates = self._load_evaluation_templates()
            print(f"   📋 Loaded {len(templates)} evaluation templates")
        
        while time.time() < end_time and not self._stop_requested:
            batch_num += 1
            batch_size = random.randint(self.config.calls_per_batch_min, self.config.calls_per_batch_max)
            
            self._run_batch(batch_size)
            
            # Print live metrics
            print(f"\r   📊 Batch {batch_num:03d} | "
                  f"Calls: {self.metrics.total_calls} | "
                  f"Success: {self.metrics.success_rate:.1f}% | "
                  f"Avg Latency: {self.metrics.avg_latency_ms:.0f}ms | "
                  f"Runtime: {self.metrics.runtime}", end="", flush=True)
            
            time.sleep(self.config.interval_seconds)
        
        # Run evaluations at the end if enabled
        if self.config.run_evaluations and templates:
            self._run_evaluations_batch(templates, self.config.evaluation_count)
        
        print(f"\n\n✅ Daemon simulation completed!")
        return self.metrics
    
    def stop(self):
        """Stop the daemon gracefully."""
        self._stop_requested = True

print("✅ DaemonRunner class defined (with evaluation support)")

In [ ]:
# Run the daemon simulation with evaluations
# Set RUN_EVALUATIONS=True to run 10 sample evaluations after simulation

RUN_EVALUATIONS = True  # Toggle to enable/disable evaluations

if not created_agents:
    print("⚠️ No agents created. Run Part 1 first.")
else:
    config = DaemonConfig(
        interval_seconds=5.0,
        calls_per_batch_min=2,
        calls_per_batch_max=4,
        threads=3,
        duration_seconds=60,        # 1 minute simulation
        run_evaluations=RUN_EVALUATIONS,
        evaluation_count=10,        # Run 10 evaluations
    )
    
    # Use the evaluation templates directory
    templates_dir = "./evaluation_templates"
    
    daemon = DaemonRunner(agents=created_agents, config=config, templates_dir=templates_dir)
    final_metrics = daemon.run()
    
    # Print final summary
    print("\n" + "=" * 70)
    print("📊 Final Simulation Metrics")
    print("=" * 70)
    print(f"   Total Calls:      {final_metrics.total_calls}")
    print(f"   Successful:       {final_metrics.successful_calls}")
    print(f"   Failed:           {final_metrics.failed_calls}")
    print(f"   Success Rate:     {final_metrics.success_rate:.1f}%")
    print(f"   Avg Latency:      {final_metrics.avg_latency_ms:.0f}ms")
    print(f"   Total Runtime:    {final_metrics.runtime}")
    
    if RUN_EVALUATIONS:
        print(f"\n🧪 Evaluation Results:")
        print(f"   Evaluation Runs:  {final_metrics.evaluation_runs}")
        print(f"   Templates Used:   4 (basic-relevance, format-check, response-completeness, semantic-similarity)")

## Part 4: Sample Evaluations

The daemon simulation can optionally run **sample evaluations** using YAML-based templates. These evaluations test agent responses for quality metrics.

### Evaluation Templates

| Template | Description | Evaluator |
|----------|-------------|-----------|
| `basic-relevance.yaml` | Checks response relevance to query | `builtin.relevance` |
| `format-check.yaml` | Validates response format | `string_check` |
| `response-completeness.yaml` | Checks if response is complete | `builtin.response_completeness` |
| `semantic-similarity.yaml` | Measures semantic similarity | `builtin.similarity` |

### How to Enable Evaluations

Set `RUN_EVALUATIONS = True` in the daemon run cell to enable evaluations.

In [ ]:
# View evaluation results from daemon run
if daemon and daemon.evaluation_results:
    print("=" * 70)
    print("🧪 Detailed Evaluation Results")
    print("=" * 70)
    
    for i, result in enumerate(daemon.evaluation_results, 1):
        status = "✅" if result.success else "❌"
        print(f"\n{i}. {status} Template: {result.template_id}")
        print(f"   Agent: {result.agent_name}")
        print(f"   Query: {result.query}...")
        print(f"   Response: {result.response[:80]}...")
        print(f"   Latency: {result.latency_ms:.0f}ms")
else:
    print("No evaluation results available. Run daemon with RUN_EVALUATIONS=True")

In [ ]:
# List all agents created in this session for portal verification
print("=" * 70)
print("📋 Agents Created for Portal Verification")
print("=" * 70)

with AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as client:
    agents = list(client.agents.list())
    fleet_agents = [a for a in agents if a.name.startswith("fleet-")]
    
    print(f"\n🔍 Found {len(fleet_agents)} fleet agents in project:\n")
    for i, agent in enumerate(fleet_agents[:10], 1):
        print(f"   {i:02d}. {agent.name} (id: {agent.id[:20]}...)")
    
    if len(fleet_agents) > 10:
        print(f"   ... and {len(fleet_agents) - 10} more")

print(f"\n💡 View these agents in Portal: {AZURE_AI_PROJECT_ENDPOINT}")

## Wrap-up

### Key Takeaways

This notebook demonstrated:

| Feature | Description |
|---------|-------------|
| **Batch Agent Registration** | Created 10 agents with different roles in a single loop |
| **Workflow Registration** | Defined 10 workflows for multi-agent orchestration |
| **Daemon Simulation** | Ran continuous traffic simulation with live metrics |
| **Sample Evaluations** | Ran 10 evaluations using YAML templates (when enabled) |
| **Portal Integration** | Agents visible in Azure AI Foundry for monitoring |

### Metrics Summary

| Metric | Description |
|--------|-------------|
| `total_calls` | Total API calls made during simulation |
| `success_rate` | Percentage of successful calls |
| `avg_latency_ms` | Average response time in milliseconds |
| `evaluation_runs` | Number of evaluations executed (if enabled) |
| `runtime` | Total simulation duration |

### Next Steps

1. **Enable Evaluations**: Set `RUN_EVALUATIONS = True` to run sample evaluations
2. **Increase Duration**: Change `duration_seconds` for longer simulations
3. **Add Tracing**: Connect Application Insights (see `1_foundry_agent_monitoring.ipynb`)
4. **Scale Up**: Increase `calls_per_batch_max` for higher load testing
5. **View in Portal**: Navigate to Azure AI Foundry to see live traces

## Additional Resources

- [Azure AI Foundry Documentation](https://learn.microsoft.com/en-us/azure/ai-services/agents/)
- [Agent Fleet Simulation Reference](https://github.com/guming3d/AI-Foundry-Agent-Simulation)
- [Azure AI Projects SDK](https://learn.microsoft.com/en-us/python/api/azure-ai-projects/)
- [Application Insights for Tracing](https://learn.microsoft.com/en-us/azure/azure-monitor/app/app-insights-overview)